In [62]:
from ortools.sat.python import cp_model
from itertools import permutations
import pandas as pd
from common import Data

In [65]:
# Read in data for current season and confirm table is accurate
df = pd.read_csv("2024.csv")
df["season"] = "2024"
standings = Data.get_nwsl_standings(df)
standings.index = standings.index - 1
standings

,team,wins,draws,losses,goals_for,goals_against,goals_diff,points
0,ORL,13,5,0,36.0,12.0,24.0,44
1,WAS,13,2,4,39.0,21.0,18.0,41
2,KC,10,5,3,42.0,28.0,14.0,35
3,NJY,10,4,4,21.0,16.0,5.0,34
4,NC,9,1,8,22.0,19.0,3.0,28
5,POR,8,3,8,29.0,27.0,2.0,27
6,CHI,7,2,9,25.0,27.0,-2.0,23
7,BFC,7,0,11,23.0,31.0,-8.0,21
8,LA,6,3,10,22.0,31.0,-9.0,21
9,RGN,5,5,9,23.0,32.0,-9.0,20


In [67]:
p_0 = list(standings["points"])
p_0

[44, 41, 35, 34, 28, 27, 23, 21, 21, 20, 19, 16, 15, 14]

In [87]:
future_games = df[df["home_score"].isna()]
n = len(standings.index)
g = [[0] * n for i in range(n)]
for i in range(n):
    for j in range(n):
        if i < j:
            i_team = standings.iloc[i]["team"]
            j_team = standings.iloc[j]["team"]
            count = len(
                future_games[
                    (
                        (future_games["home"] == i_team)
                        & (future_games["away"] == j_team)
                    )
                    | (
                        (future_games["away"] == i_team)
                        & (future_games["home"] == j_team)
                    )
                ].index
            )
            g[i][j] = count
            g[j][i] = count
g

[[0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1],
 [1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 1],
 [0, 1, 0, 0, 0, 0, 1, 2, 1, 0, 1, 2, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 2, 0, 0, 0, 0, 1, 1, 0, 0, 1],
 [0, 1, 0, 0, 1, 2, 0, 0, 0, 1, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 2],
 [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0],
 [0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0],
 [1, 1, 0, 1, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0]]

In [91]:
# Creates the model.
model = cp_model.CpModel()

####################################################
# Creates the variables.

# Team we are checking to see if they clinched
k = 1
# w[i][j] represents the number of wins of i over j
w = [
    [model.new_int_var(0, 2, f"w_{i}_{j}") if i != j else None for j in range(n)]
    for i in range(n)
]
# t[i][j] represents the number of ties between i and j
t = [
    [model.new_int_var(0, 2, f"t_{i}_{j}") if i != j else None for j in range(n)]
    for i in range(n)
]
# p[i] represents the number of points team i has
p = [model.new_int_var(0, 3 * 26, f"p_{i}") for i in range(n)]
# b[i] is 1 if team i has more points than team k
b = [model.new_bool_var(f"b_{i}") for i in range(n)]

####################################################
# Creates the constraints.

# Wins must sum up to the total games
for i in range(n):
    for j in range(n):
        if i < j:
            model.add(w[i][j] + w[j][i] + t[i][j] <= g[i][j])

# Ties must be reflexive
for i in range(n):
    for j in range(n):
        if i != j:
            model.add(t[i][j] == t[j][i])

# Points calculation
for i in range(n):
    if i == k:
        model.add(p[i] == p_0[i])
    else:
        model.add(
            p[i]
            == p_0[i]
            + 3 * sum(w[i][j] for j in filter(lambda x: x != i, range(n)))
            + 1 * sum(t[i][j] for j in filter(lambda x: x != i, range(n)))
        )


# Calculating if a team is better than team k
for i in range(n):
    model.add(p[i] > p[k]).only_enforce_if(b[i])

####################################################
# Set the optimization function.
model.maximize(sum(b[i] for i in range(n)))

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"sum={sum(solver.value(b[i]) for i in range(n))}")
else:
    print("No solution found.")

sum=8
